### python描述器！！(https://docs.python.org/zh-cn/3.9/howto/descriptor.html#simple-example-a-descriptor-that-returns-a-constant)

1. 定义了 __get__()，__set__() 或 __delete__() 其中任何一个的对象
还可以有 __set_name__() 方法
2. 描述器仅在用作类变量时起作用。放入实例时，它们将失效
3. 属性查找期间，描述器由点运算符调用
4. 描述器被分配给类字典中的公开属性
5. 在Python中，实例的实际数据和属性存储在实例的__dict__属性中
    1. 类的属性存储在类的__dict__属性中
    2. 类的属性可以被访问，但是默认情况下，实例的属性是私有的，也就不属于公开属性，实例字典保存了实例的属性和实际数据，需要通过实例方法来访问
    3. 也就是类字典不包括实例属性和实际数据
6. 允许存储在类变量中的对象控制在属性查找期间发生的情况
7. 定义了 __set__() 或 __delete__()，则它会被视为数据描述器。 仅定义了 __get__() 的描述器称为非数据描述器

In [11]:
"""
__get__
注意参数
"""
class Ten:
    def __get__(self, obj, objtype=None):
        """
        self:y
        :param obj: a
        :param objtype: 
        :return: 常量
        """
        return 10
class A:
    def __init__(self,name):
        self.name = name#实例属性
    x = 5                       
    y = Ten() 
a=A("ghj")
print(a.__dict__)#实例字典
print(A.__dict__)
print(a.x)#点运算符会找到存储在类字典中的键 x 及对应的值 5

print(a.y)#点运算符会根据描述器实例的 __get__ 方法将其识别出来，调用该方法并返回 10


{'name': 'ghj'}
{'__module__': '__main__', '__init__': <function A.__init__ at 0x000001A2DA964310>, 'x': 5, 'y': <__main__.Ten object at 0x000001A2DA97ACD0>, '__dict__': <attribute '__dict__' of 'A' objects>, '__weakref__': <attribute '__weakref__' of 'A' objects>, '__doc__': None}
5
10


8. 属性访问的默认行为
    1. 对于实例来说，a.x 的查找顺序会从 a.__dict__['x'] 开始
    2. 然后是 type(a).__dict__['x'](即他的类变量等等)
    3. 接下来依次查找 type(a) 的方法解析顺序（MRO）
    4. 如果找到的值是定义了某个描述器方法的对象，转而发起调用描述器方法
    5.  实例查找通过命名空间链进行扫描，数据描述器的优先级最高，其次是实例变量、非数据描述器、类变量，最后是 __getattr__() （如果存在的话）。
        1. 当实例中存在一个数据描述器时，无论是否存在同名的实例变量、非数据描述器或类变量，数据描述器都会优先被访问。
        2. 实例变量：如果实例中存在同名的实例变量，且没有数据描述器覆盖该属性，则实例变量会被优先访问
        3. 非数据描述器：非数据描述器是指只实现了 __get__() 方法的描述器。如果实例中不存在同名的实例变量，但存在一个非数据描述器，则该非数据描述器会被访问。
        4. 类变量：如果实例中既没有同名的实例变量也没有非数据描述器，Python 将在类中查找同名的属性。如果类中存在同名的属性（类变量），则该类变量会被访问。
                
    6. 对于下面的例子，由于访问pete.name,在pete.__dict__中没有找到(因为已经是私有名称了)，然后再类中找到了name类变量，即描述器，调用__get__方法，返回了值

9. !!!数据和非数据描述器的不同之处在于，如何计算实例字典中条目的替代值。如果实例的字典具有与数据描述器同名的条目，则数据描述器优先。如果实例的字典具有与非数据描述器同名的条目，则该字典条目优先。
    1. 解释：如果有__set__方法，那么就会在实例字典中创建私有名称，替代同名的条目，如果没有，只有__get__，则不会改变

In [ ]:
"""
托管属性
1. 托管对实例数据的访问
2. 由于name和age属性已经在Person类中使用描述符绑定了，因此当我们在__init__方法中设置self.name和self.age时，实际上是调用了描述符的__set__方法来设置属性的值。这样描述符就可以在访问和设置属性时进行日志记录。
3. 
"""
from loguru import logger
import sys
logger.remove()
logger.add(sys.stderr)
class log:
    def __set_name__(self, owner, name):
    #owner=类，name=类变量的名称
        self.public_name = name#给外部用户的名称
    #防止与描述符的公共名称发生冲突
        self.private_name = '_' + name#内部维护的名称

    def __get__(self, obj, objtype=None):
        #传入的参数已经固定
        value = getattr(obj, self.private_name)
        #因为初始化后，实例字典已经是private_name，所以要获取的话只能用private_name
        logger.debug("访问")
        return value

    def __set__(self, obj, value):
        logger.debug("改")
        setattr(obj, self.private_name, value)
        #初始化，或者更改时调用，操作结果储存到实例的实例字典
        #print(vars(pete))和print(pete.__dict__)都只有私有名称
        #但是公共名称还是在类中的属性name当中
class Person:

    name =log()                # First descriptor instance
    age =log()                 # Second descriptor instance
#这步完成之后， def __set_name__(self, owner, name):已经自动调用并且传参
    def __init__(self, name, age):
        self.name = name                 # Calls the first descriptor
        self.age = age                   # Calls the second descriptor

    def birthday(self):
        self.age += 1

#print(vars(vars(Person)['name']))
pete = Person('Peter P', 10)
print("name" in dir(pete))
print("_name" in dir(pete))
print("age" in dir(pete))
print("_age" in dir(pete))
print("name" in pete.__dict__)
print("age" in pete.__dict__)
print("_age" in pete.__dict__)
print("_name" in pete.__dict__)

print(pete.name)

#描述符会在类的字典中创建一个私有的属性名称，以便存储属性的值
print(Person.__dict__)
print(vars(vars(Person)['name']))


需要的概念：
1. 方法
2. 函数
3. 对象

In [4]:
"""
函数：
1. 外部的函数
2. 通过类直接调用函数
"""
def add():
    pass
class T():
    def mul(self):
        pass
print(add)
print(T.mul)

<function add at 0x000001A2D8F85B80>
<function T.mul at 0x000001A2DA3DFE50>


In [1]:
"""
方法(绑定方法)
1. 类实例调用
"""
class T():
    def mul(self):
        pass
print(T.__dict__)#类词典
t=T()
print(t.mul)

{'__module__': '__main__', 'mul': <function T.mul at 0x000001E4866593A0>, '__dict__': <attribute '__dict__' of 'T' objects>, '__weakref__': <attribute '__weakref__' of 'T' objects>, '__doc__': None}
<bound method T.mul of <__main__.T object at 0x000001E4865BD130>>


4. 背后的原理
    1. 实例调用类的函数时，存储在类词典中的函数将被转换为方法。方法与常规函数的不同之处仅在于对象实例被置于其他参数之前。方法与常规函数的不同之处仅在于第一个参数是为对象实例保留的。按照惯例，实例引用称为 self
    2. 实例访问类中的函数时，函数被实例调用，自动把这个实例作为参数，在描述器里面自动执行__get__方法，返回一个方法
    3. 没有显示的写出描述器，他们是隐式的吗，在函数内部
   

In [14]:
"""
函数与方法的绑定等价实现
"""

#即函数的__get__方法的等价实现，实现了一个类装饰器
class Methodtype:
    def __init__(self,func,obj):
        self.func = func
        self.obj = obj
        #因为返回的方法也是可调用的，实现__call__方法，是内置方法
        #用callable判断是不是可调用的
    def __call__(self,*args,**kwargs):
        func=self.func
        obj=self.obj
        #解释了为什么第一个参数是self或者是cls
        return func(obj,*args,**kwargs)



    #实验：将一个函数转化为方法，即实现<函数名>()的调用方式
class Calculator:
    #此时的add函数只能通过实例调用或者类调用
    def add(self, x, y):
        return x + y
# 创建一个Calculator的实例对象
calculator = Calculator()
#返回一个方法
add_method = Methodtype(Calculator.add,calculator)
#实现成功
result = add_method(3, 4)
print(result)


#自动__get__的实现
class Function(object):
    def __get__(self, obj, objtype=None):
        #这里的 objtype可以是对象/类/超类/。。。。
        if obj is None:
            return self
#调用MethodType(self, obj)时实际上是在创建一个MethodType类的实例
        return Methodtype(self, obj)


"""
一般会自动创建方法
每个函数都有__get__描述器方法，以便在属性访问时绑定其为方法。
这意味着函数其是非数据描述器，它在通过实例进行点查找时返回绑定方法
"""

7


'\n一般会自动创建方法\n每个函数都有__get__描述器方法，以便在属性访问时绑定其为方法。\n这意味着函数其是非数据描述器，它在通过实例进行点查找时返回绑定方法\n'

In [7]:
"""
又是一个等价实现！！！像绑定属性一样，绑定函数,用类装饰器
"""
class FunctionMethod:
    def __init__(self, func):
        self.func = func
    def __get__(self,obj,objtype=None,*args,**kwargs):
        print("实际已经调用这里了")
        #判断是什么类型
        #描述器的 __get__ 方法中，通常会根据 obj 参数的值来决定返回什么。
        # 如果 obj 是 None，表示这个方法是通过类访问的，应该返回描述器本身。
        # 否则，应该返回一个新的方法对象
        if obj is None:
            print("通过类调用了")
            return self
            #return self.func则返回传入的函数
        print("通过实例调用了")
        return self.func(obj,*args,**kwargs)
    
    def __call__(self, *args, **kwargs):
        pass
class MyClass():
    @FunctionMethod
    def my_method(self):
        print("my_method")
        #return "hello"

print(MyClass.my_method)
print("next")
#<__main__.FunctionMethod object at 0x000001AADD80A0D0>
# 返回了描述器本身，在这里就是返回了装饰器

my_class=MyClass()
#<bound method MyClass.my_method of <__main__.MyClass object at 0x0000022263F0A190>>
print(my_class.my_method)
print("next")



实际已经调用这里了
通过类调用了
next
实际已经调用这里了
通过实例调用了
my_method
None
next


In [2]:
"一个需要的注意"
def add(x,y):
    print(x+y)
def test():
    return add(1,2)
test()
#之所以输出结果是3和None，是因为test()输出是3，在print他结果就是None
print(test())
print("next")
def add_1(x,y):
    return x+y
def test_1():
    return add_1(1,2)
test_1()
#之所以输出结果是3和None，是因为test()输出是3，在print他结果就是None
print(test_1())

3
3
None
next
3


super的用法
(https://rhettinger.wordpress.com/2011/05/26/super-considered-super/)
(https://www.youtube.com/watch?v=X1PQ7zzltz4&list=PLNZHF9CMm7P1CBKcn8I2vtZB487FNUPKm&index=16)

1. 一个关键的区别就是新式类能够从python的内置类型中继承，而经典类不行
2.  C3 线性化算法，广度优先，MRO
3. MRO不只是对方法的搜索，也是对属性的搜索

使用super时的建议
1. 使用固定参数-保证传入的参数正确
2. 对于一个方法，只要有一个实现了super，其他的父类也要实现，但是最后的Root可以中断-保证了一定调用父类的对应方法
3. 使用参数的分割，关联super和类——关键字分离
4. super实际也是一个类，调用super实际上创建了当前类第的一个代理对象，自动传入self参数，注意，与已经创建的当前类的对象不同
5. 用mro查看顺序
6. 通过实例的super和通过类的super
7. 层次结构中的类都必须继承自某个Root类，比如A继承自B,C，还有一个Root类，此时B,C必须继承Root
8. 

### 新式类主要默认继承了object,查看他实现的方法

In [ ]:
print(dir(object))

###### 类中的内置方法,类可以通过定义具有特殊名称的方法来实现由特殊语法调用的某些操作
## 即魔术方法
有些方法之所以不用显示的写出，是因为他们自动的使用了父类object的方法

__del__ 方法,用于在对象被销毁（即垃圾回收）时执行一些清理操作
不需要再函数中显示写出,也可以删除类中的方法,属性

In [3]:
class MyClass:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def add(self,a,b):
        return a+b
# 创建一个对象
obj = MyClass(1,2)

print(obj.x)
del obj
print(obj.x)
del obj.add
# 删除对象的引用，触发对象销毁

1


NameError: name 'obj' is not defined

__repr__ 方法,返回表示对象的字符串，通常用于调试和打印对象时使用,需要写出
可以定义输出结果的方式,直接打印对象时才调用

In [1]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def  __eq__(self, other): 
        return self.x == other.x and self.y == other.y
    def __repr__(self):
        return f"Point({self.x}, y={self.y})"
    
# 创建一个 Point 对象
point = Point(1,2)
point_two=Point(3,4)
# 调用 __repr__ 方法
print(repr(point))
print(point.__eq__(point_two))
print(point_two==point)


Point(1, y=2)
False
False


__len__ 方法,返回对象的长度，用于内置函数 len() 的调用
因为此时len函数的参数变成了类的对象，在len内部，已经定义了要测量长度的实例的属性

In [8]:
class MyList:
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data)
    

# 创建一个 MyList 对象
my_list = MyList([1, 2, 3, 4, 5])
# 调用 __len__ 方法
print(len(my_list))  # 输出 5


5


__getitem__ 方法,用于实现对象索引操作,高度可自定义

In [9]:
class MyList:
    def __init__(self, data):
        self.data = data

    def __getitem__(self, index):
        return self.data[index]

my_list = MyList([1, 2, 3, 4, 5])
print(my_list[2])


3


实际上就是实现了列表List类的方法，只不过可以自定义了
__setitem__(self, key, value): 设置对象的元素，支持通过索引或键来设置元素的值
__delitem__(self, key): 删除对象的元素，支持通过索引或键来删除元素。
__contains__(self, item): 判断对象是否包含某个元素。通常用于实现成员关系测试操作符 in。

In [ ]:
class MyList:
    def __init__(self, data):
        self.data = data

    def __setitem__(self, index, value):
        self.data[index] = value
    def __delitem__(self, index):
        del self.data[index]
    def __contains__(self, item):
         return item in self.data
    def __iter__(self):
        return iter(self.data)
my_list = MyList([1, 2, 3, 4, 5])
my_list[2] = 10
del my_list[1]
print(my_list.data)

print(10 in my_list)
for item in my_list:
    print(item)

__getattr__(self, name)小心无限循环
1. 用于在访问不存在的属性时进行处理，如果没有找到某个属性，它可以提供一个默认的值或者执行其他自定义的逻辑
2. (https://www.cnblogs.com/huchong/p/8287799.html)


In [4]:
class ClassA(object):

    def __init__(self, classname):
        self.classname = classname

    def __getattr__(self, attr):
        #还可以加函数什么的，定义一个静态方法
        return('invoke __getattr__', attr)


insA = ClassA('ClassA')
print(insA.__dict__) # 实例insA已经有classname属性了
# {'classname': 'ClassA'}

print(insA.classname) # 不会调用__getattr__
# ClassA

print(insA.grade) # grade属性没有找到，调用__getattr__
# ('invoke __getattr__', 'grade')


{'classname': 'ClassA'}
ClassA
('invoke __getattr__', 'grade')


__getattribute__小心无限循环
__getattribute__方法一直会被调用，无论属性name是否追溯到
不能访问值！只有记录

In [5]:
class ClassA(object):

    def __init__(self, classname):
        self.classname = classname

    def __getattribute__(self, attr):
        #return super().__getattribute__(attr)
        return('invoke __getattribute__', attr)


insA = ClassA('ClassA')
print(insA.__dict__)
# ('invoke __getattribute__', '__dict__')

print(insA.classname)
# ('invoke __getattribute__', 'classname')

print(insA.grade)
# ('invoke __getattribute__', 'grade')

('invoke __getattribute__', '__dict__')
('invoke __getattribute__', 'classname')
('invoke __getattribute__', 'grade')


__setattr__小心无限循环
1. 拦截所有属性的的赋值语句
2. self.arrt = value 就会变成self.__setattr__("attr", value)
3. 如果类自定义了__setattr__方法，当通过实例获取属性尝试赋值时，就会调用__setattr__
4. 自己实现__setattr__有很大风险，一般情况都还是继承object类的__setattr__方法。

In [ ]:
#object的__setattr__方法
class ClassA(object):

    def __init__(self, classname):
        self.classname = classname

insA = ClassA('ClassA')

print(insA.__dict__)
# {'classname': 'ClassA'}

insA.tag = 'insA'    

print(insA.__dict__)
# {'tag': 'insA', 'classname': 'ClassA'}


In [ ]:
#自己的定义
class ClassA(object):
    def __init__(self, classname):
        self.classname = classname

    def __setattr__(self, name, value):
        # self.name = value  # 如果还这样调用会出现无限递归的情况
        print('invoke __setattr__')

insA = ClassA('ClassA') # __init__中的self.classname调用__setattr__。
# invoke __setattr__

print(insA.__dict__)
# {}

insA.tag = 'insA'    
# invoke __setattr__

print(insA.__dict__)
# {}

__new__方法(https://www.youtube.com/watch?v=-zsV0_QrfTw)
1. 创建实例时，__init__和__new__都会被调用
2. new负责创建和返回实例对象——所以定义的new方法会有return——一个对象，而init负责初始化，设置默认值等等——没有return
3. new是一个类方法，接受要返回的实例的类，也就是要创建A的实例，则传入A
4. 可以在创建对象也就是在__new__方法中，做一些对数据的改变
5. 这样的写法是高性能的，对于python中C的扩展，尽量编写少的python语言，提高性能

In [7]:
class A:
    def __new__(cls, a, b):
        """
        一个类方法
        :param args: 
        :param kwargs: 
        """
        a = a + 1
        print("new_here", cls, a, b)
        #return——返回实例对象，如果去掉这行，则__init__不会调用
        return super().__new__(cls)
    
    def __init__(self,*args,**kwargs):
        print("__init__，初始化", self, args, kwargs)

I=A(1, b=8)



new_here <class '__main__.A'> 2 8
__init__，初始化 <__main__.A object at 0x000002B29C05D040> (1,) {'b': 8}


cls 和 self cls表明可以访问类的属性和方法，self 访问实例的属性和方法，

In [9]:
class MyClass:
    '类属性'
    calss_num = 8
    class_variable = "I am a class variable"
    def __init__(self,instance_variable):
        "实例属性，cls 不可以访问,但是可以通过MyClass.class_variable访问类属性,MyClass.class_method()访问类方法"
        self.instance_variable = instance_variable
    @classmethod
    def class_method(cls):
        print("This is a class method")
        print("Accessing class variable:", cls.class_variable)
    def instance_method(self):
        print("This is an instance method")
        print("Accessing instance variable:", self.instance_variable)
    @classmethod
    def add(cls, a, b):
        return  a+b
    @classmethod
    def multi(cls,c):
        return c*cls.add(2,2)*cls.calss_num

print(MyClass.class_method())

instance=MyClass("ghj")

print(instance.instance_method())

print(MyClass.multi(5))

This is a class method
Accessing class variable: I am a class variable
None
This is an instance method
Accessing instance variable: ghj
None
160


以下划线开头的一般为私有变量，不能直接访问。
porperty装饰器，加上之后只需要先实例化如：p ，然后使用p.prop就相当于调用方法，可以用@prop.setter操控属性的值等等
不需要在调用时使用括号
创建只读属性,用prop.setter和prop.deleter变为可写功能
读写方法名必须上下一致

In [10]:
class Prop():
    def __init__(self,num):
        self._nums=num
    @property
    def prop(self):
        return self._nums

    @prop.setter
    def prop(self,value):
        if value>=0:
            self._nums=value
        else:
            raise ValueError("必须大于等于0")
        
    @prop.deleter
    def prop(self):
        print("删除属性")
        del self._nums
p=Prop(5)
print(p.prop)
p.num=10
print(p.prop)
del  p.prop

5
10
10
删除属性


静态方法，使用@staticmethod装饰器，静态方法不需要实例化，直接类名.方法名()调用,他只是在类里，但是没有明确的关系，不需要传入self或者cls等第一个参数，他与类是独立的，不访问类的属性或方法
静态方法还可以创建实例调用，同时传入参数，此时实例不算做参数传入
类方法，使用@classmethod装饰器，类方法需要传入cls参数，cls代表类本身，他可以访问类属性，第一个参数是cls,可以访问类属性类方法，包括@staticmethod方法

In [3]:
class MyClass_one:
    test_1=10
    test_2=20
    @staticmethod
    def add_two(a,b):
        return a+b
    @classmethod
    def multi_two(cls,a,b):
        return a*b*cls.add_two(5,6)
    #体现静态方法与类的隔离的示例
    @staticmethod
    def add(x):
        return x+test_1  #NameError: name 'test_1' is not defined
print(MyClass_one.add_two(2,3))
print(MyClass_one.multi_two(2,3))
print(MyClass_one.add(1))

5
66


NameError: name 'test_1' is not defined

LEGB法则： Python会按照优先级依次搜索4个作用域，以此来确定该变量名的意义。首先搜索局部作用域(L)，之后是上一层嵌套结构中def或lambda函数的嵌套作用域(E)，之后是全局作用域(G)，最后是内置作用域(B)。按这个查找原则，在第一处找到的地方停止。如果没有找到，则会出发NameError错误

1. 类的dataclass装饰器，使用@dataclass装饰器，会自动生成__init__,__repr__,__eq__,__ne__,__lt__,__le__,__gt__,__ge__,__hash__等方法
2. 默认情况下，dataclass创建的实例是可变的，但你可以通过frozen=True参数使其成为不可变的：
3. 不需要定义__eq__和__lt__方法，因为当order = True被调用时，dataclass 装饰器会自动将它们添加到我们的类定义中
后初始化处理__post_init__,生成的__init__方法在返回之前 调用__post_init__返回。因此，可以在函数中进行任何处理
4. 如果在继承子类中要连续调用父类的后期初始化方法，需要用到super()
default_factory必须是一个可以调用的 无参数方法（通常为一个函数）
5. MRO是Method Resolution Order的缩写，指的是在多重继承中确定方法调用顺序的算法（C3和深度优先）(http://www.cnblogs.com/btchenguang/archive/2012/09/17/2689146.html)

In [28]:
from dataclasses import dataclass,field
from typing import ClassVar,List
"""给装饰器添加参数"""
def num():
    return [x for x in range(1,10)]
@dataclass(frozen=False,init=True,repr=True,order=True)
class test_two():
    name:str
    age:int
    class_var:ClassVar[int]=100
    #score:List[int]=field(default_factory=num)
    """添加自定义方法"""
    @classmethod
    def global_var(cls):
        return cls.class_var
#print(test_two("ghj",19).score)
ins=test_two("ghj",19)
print(ins.class_var)
print(ins.age<ins.class_var)
print(ins.global_var())

[1, 2, 3, 4, 5, 6, 7, 8, 9]
100
True
100


如果通过__post_init__初始化,在这个例子中要传入一个空列表，不如field方便

In [ ]:

@dataclass(frozen=False,init=True,repr=True,order=True)
class test_three(test_two):
    hobby:str
    grade:List[int]=10
    def __post_init__(self):
        self.finall_score=self.grade+self.score
instance=test_three("ghj",age=19,hobby="yut")
print(instance.finall_score)

dataclass继承dataclass

hasattr用于检查一个对象是否具有指定的属性。它接受两个参数，第一个参数是要检查的对象，第二个参数是要检查的属性名

In [10]:
class MyClass_one:
    num=8
    #换成静态方法和实例方法，类方法都可以检测
    def add_two(self,a,b):
        return a+b
print(hasattr(MyClass_one,'num'))
print(hasattr(MyClass_one,'add_two'))

True
True


__new__

抽象类
1. Python默认不支持抽象类和抽象方法，因此有一个名为ABC（abstract baseclasses）的包，通过它我们可以将类或方法抽象化
2. 当一个类继承ABC类，并且是至少有一个抽象方法，才是抽象类
3. 继承抽象类的类，如果没有定义父类中存在的所有抽象方法，那么子类也会自动成为抽象类
4. 抽象类无法实例化

In [1]:
#4
from abc import ABC,abstractmethod
class T(ABC):
    @abstractmethod
    def test(self):
        pass
print(T.__dict__)
t=T()

{'__module__': '__main__', 'test': <function T.test at 0x000001EA7B6F6EE0>, '__dict__': <attribute '__dict__' of 'T' objects>, '__weakref__': <attribute '__weakref__' of 'T' objects>, '__doc__': None, '__abstractmethods__': frozenset({'test'}), '_abc_impl': <_abc._abc_data object at 0x000001EA7B6F76C0>}


TypeError: Can't instantiate abstract class T with abstract method test

In [ ]:
#3
from abc import ABC, abstractmethod
class Person(ABC):
    @abstractmethod
    def add(self):
        pass
    @abstractmethod
    def mul(self):
        pass
class Student(Person):
    def mul(self):
        return 5
    def add(self):
        return 7
A=Student()